In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv',sep='\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
# Vectorizing the raw text information

df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [5]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df['message']

In [8]:
y = df['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count_vect = CountVectorizer()

In [12]:
# Fit the vectorizer to the data
# count_vect.fit(X_train)
# X_train_counts = count_vect.transform(X_train)

# Transform the original text message --> Vector

X_train_counts = count_vect.fit_transform(X_train)

In [13]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [14]:
X_train.shape

(3733,)

In [15]:
X_train_counts.shape

(3733, 7082)

In [16]:
# Giving more important words more weight

from sklearn.feature_extraction.text import TfidfTransformer

In [17]:
tfidf_transformer = TfidfTransformer()

In [18]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [19]:
X_train_tfidf.shape

(3733, 7082)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
vectorizer = TfidfVectorizer()

In [22]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [23]:
from sklearn.svm import LinearSVC

In [24]:
clf = LinearSVC()

In [25]:
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [26]:
# Combine everything into one single pipeline step

from sklearn.pipeline import Pipeline

In [27]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [28]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [29]:
predictions = text_clf.predict(X_test)

In [31]:
from sklearn.metrics import confusion_matrix,classification_report

In [32]:
# Much better results

print(confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [33]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [34]:
# Also check out accuracy at 98%

from sklearn import metrics

In [35]:
metrics.accuracy_score(y_test,predictions)

0.989668297988037

In [36]:
# The text classification pipeline can predict ham

text_clf.predict(["Hi how are you today?"])

array(['ham'], dtype=object)

In [37]:
# the pipeline can also predict spam

text_clf.predict(["Congratulations! You've been selected as a winner. Text Won to 44255 congratulations free entry to contest."])

array(['spam'], dtype=object)